In [66]:
%run kart-numpy.py

In [18]:
%run bwa.py

## Test(Past version)

In [45]:
with open("data/SRR11528307_SarsCov2.fna", "r") as f:
    ref = "".join([l.strip() for l in f.readlines()][1:-4])
                            
# TCCGGGACGCTCAGTAATGTGACGA GACGATAGCTGAAAAC
read_1 = 'GTAATAAGACAGTAGGTGAGTTAGGTGATGTTAGAGTAAACAATGAGTTACTTGTTTCAACATGCCAATTTAGATTCTTGCAAAAGAGTCTTGAACGTGGTGTGTAAAACTTGTGGACAACAGCAGACAACCCTTAAGGGTGTAGAAG'

In [46]:
print(len(ref))
print(len(read_1))
# print(ref)

29600
148


In [47]:
bwa_run(read_1, ref, tolerance=0)

[]

In [48]:
A = LMEM_explore(read_1, ref, k=10, verbose=1)

> Finding simple pairs (0.00%)
> Finding simple pairs (24.32%)
> Finding simple pairs (25.00%)
> Finding simple pairs (100.00%)


In [49]:
clustered_A = form_cluster(A, g=10)
print(clustered_A)
clustered_A = form_candidate_regions(clustered_A)
print("Clustered:")
for i in clustered_A:
    print(i)
pprint_align(clustered_A, read_1, ref)

[[[0, 35, 5269, 5304, 36], [37, 147, 5305, 5415, 111]]]
Clustered:
[[0, 35, 5269, 5304, 36], [36, 36, -1, -1], [37, 147, 5305, 5415, 111]]
Candidate 1:
Index:     5269                                                                                                                                                  5417
Reference: GTAATAAGACAGTAGGTGAGTTAGGTGATGTTAGAG-AAACAATGAGTTACTTGTTTCAACATGCCAATTTAGATTCTTGCAAAAGAGTCTTGAACGTGGTGTGTAAAACTTGTGGACAACAGCAGACAACCCTTAAGGGTGTAGAAG
Read:      GTAATAAGACAGTAGGTGAGTTAGGTGATGTTAGAGTAAACAATGAGTTACTTGTTTCAACATGCCAATTTAGATTCTTGCAAAAGAGTCTTGAACGTGGTGTGTAAAACTTGTGGACAACAGCAGACAACCCTTAAGGGTGTAGAAG
Aligned:   ************************************ ***************************************************************************************************************



# Test (full version)

In [19]:
bwa(ref_path,read_path, num_reads=5)

ATGTCACAGCGTCCTAGATGGTGTCCA
--read
[]
TCACGACATTGGTAACCCTAAAGCTATTAAGTGTGTACCTCAAG
--read
[18877]
AATCAAAAAGAGTTGATTTTTGTGGAAAGGG
--read
[24520]
CCATTCTACATCAGCTTGAGGTACACACTTAATAGCTTTAGGGTTACCAATGTCGTGAAGAACTGGGAATTTGTCTGCTAATAATGCAGCTTTAACAACCATGTGTTGAACCTTTCTACAAGCCGCATTAAT
--read
[]
TGTATCACAATTAACACAATTCCAATTGTGTA
--read
[]


In [37]:
reads = []

In [44]:
read_path = r'data/SRR11528307_R1.fastq'
ref_path = r'data/SRR11528307_SarsCov2.fna'

raw_reads = get_n_reads(read_path, num_reads=100)
ref = get_ref(ref_path)

DELETE = 2
PERTURB = 2

for r in raw_reads:
    if len(r) < 30:
        continue
    
    print("original:", r)
    if len(bwa_run(r, ref)) != 0:
        for _ in range(DELETE):
            idx = random.randint(0, len(r)-1)
            r = list(r)
            del r[idx]
            r = "".join(r)
        for _ in range(PERTURB):
            idx = random.randint(0, len(r)-1)
            r = list(r)
            r[idx] = "A" if r[idx] != "A" else "T"
            r = "".join(r)
        print("altered:", r)
        reads.append(r)

original: ACCAAAGCACTCACAGTGTCAACAATTTCAGCAGGACAACGCCGACAAGTTCCGAGGAACATGTCTGGACCTATAGTTTTCATAAGTCTACACACTGAATTGAAATATTCTGGTTCTAGTGTGCCCTTAG
original: TGTCATGATGGAAAAGCACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAATGGCACACACTGGTTTGTAACACAAAGGAA
altered: TATCATGATGGAAAAGACACTTTCCTCGTGAAGGTGTCTTTGTTTCAAATGGCACACACTGGTTTGTATCACAAAGAA
original: AGAGCCTAAAAAGGACAAAAAGAAGAAGGCTGATGAAACT
altered: AGAGCCTAAAAAAGACAAAAAAGAAGGCTGATGAAACA
original: TTTAATACCCTTATATTTACGCTGTATAGTTGAAACTATGGCTTTAGTTTCCACACAGACAGGCA
original: CTCGTTCCTCATCACGTAGTCGCAACAGTTCAAGAAATTCAACTCCAGGCAGCAGTAGGGGAACTTCTC
altered: CTCGTTCCTCATCACGTATCGCAACAGTTCAATAATTCAACTCCAGGCAACAGTAGGGGAACTTCTC
original: AAGTGGTCCATTAGTAGCTATGTAATCATCAGATTCAACTTGCATGGCATTGTTAGTAGCCTTATTTAAGGCTCCTGCAACACCTCCTCCAT
original: GCCGCAACACAGGCGAACTCATTTACTTCTGTACCGAGTTCAACTGTATAGGCAGAGCACCTCTCATTAA
original: CTCATTCAAGGAGGAGTTAGATAAATATTTTAAGAATCATACATCACCAGATGTTGATTTAGGTGACATCTCTGGC
altered: CTCATTCAAGGATGAGTAGATAAATATTTTAGAATCATACATCACCAGATGTTGATTTAGGTGACATC

In [46]:
reads = reads[:100]

In [47]:
len(reads)

100

In [48]:
import pickle
with open("100_reads_perturbed.pt", "wb") as f:
    pickle.dump(reads, f)

In [50]:
with open("100_reads_perturbed.pt", "rb") as f:    
    reads = pickle.load(f)

In [78]:
%run kart.py

In [81]:
%run kart_numpy.py

In [100]:
%run kart_multiprocessing.py

In [ ]:
import time

start = time.time()
kart_exp(reads, ref_path, k=10, g=10, num_reads=1, verbose=0)
end = time.time()
print("Time spent:", end-start)

start = time.time()
kart_exp(reads, ref_path, k=10, g=10, num_reads=5, verbose=0)
end = time.time()
print("Time spent:", end-start)

start = time.time()
kart_exp(reads, ref_path, k=10, g=10, num_reads=10, verbose=0)
end = time.time()
print("Time spent:", end-start)

start = time.time()
kart_exp(reads, ref_path, k=10, g=10, num_reads=20, verbose=0)
end = time.time()
print("Time spent:", end-start)

naive kart: 95.52167773246765, 558.2398657798767, 740.8427379131317, 2008.9000990390778

numpy kart: 56.3682918548584, 435.8727900981903, 577.298269033432, 1334.874221086502

multiprocessing_pool kart: 54.42660307884216, 276.7442009449005, 435.26404094696045, 699.4486169815063

mpi4py kart: 43.1665472829466, 293.7563929448473, 430.35481786727905, 719.9554629325867

In [124]:
%%writefile mpi_script.py

from kart_numpy_mpi import split_reads, LMEM_explore, form_cluster, form_candidate_regions, pprint_align, select_cr, assemble_genome
from bwa import get_ref
import random
import pickle
import time
from mpi4py import MPI

comm = MPI.COMM_WORLD
num_workers = comm.Get_size()
rank = comm.Get_rank()

if rank==0:
    with open("100_reads_perturbed.pt", "rb") as f:    
        reads = pickle.load(f)
    start = time.time()
    read_data = split_reads(reads[:20], num_workers) # TODO split_reads into sublists

else:
    read_data = None 

read_data_chunk = comm.scatter(read_data, root=0)

ref = get_ref(r'data/SRR11528307_SarsCov2.fna')
for read in read_data_chunk:
    A = LMEM_explore(read, ref, k=10, verbose=0)
    clustered_A = form_cluster(A, g=10)
    crs = form_candidate_regions(clustered_A)
    cr = select_cr(crs, read, ref)
mapped_list = comm.gather(cr, root=0)

if rank==0:
    print(mapped_list)
    end = time.time()
    print("Time spent:", end-start)

Overwriting mpi_script.py


In [125]:
!mpiexec -n 10 --oversubscribe python mpi_script.py 

[(25822, 25958), (14309, 14362), (19928, 20049), (11573, 11666), (19150, 19278), (27357, 27458), (9627, 9762), (8722, 8858), (19799, 19847), (16270, 16370)]
Time spent: 719.9554629325867
